In [ ]:
#ls

In [ ]:
cd 'RUN2'

/content/drive/My Drive/RUN2


In [ ]:
#!unzip admset213.zip 

In [ ]:
cd admrun2

/content/drive/My Drive/RUN2/admrun2


In [ ]:
ls

admrun2/       test_images_from_train/  train_sample.csv
admset213.zip  test_sample.csv          validation_images_model/
run2.ipynb     train.csv                validation_sample.csv
test.csv       train_images_model/


In [ ]:
import numpy as np 
import pandas as pd 
import sys, requests, shutil, os

data_train=pd.read_csv("train_sample.csv")
data_valid=pd.read_csv("validation_sample.csv")
data_test=pd.read_csv("test_sample.csv")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
#from tensorflow.keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
import csv
import os
#import cv2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import math
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import tensorflow as tf
import tensorflow.keras

In [ ]:
'''
        There are few images that are not possible to open because they are corrupted or
    they do exhibit some properties which are not applicable to standard algorithms, 
    these may result in algorithm going to infinite loop trying to process it.All those images 
    are removed using the function below. 
'''
from PIL import Image
import os

def filter_images(dir):
    i = 1000
    count = 0
    while i <= 1000:
        f = str(i)
        for root, dirs, files in os.walk(dir +'/'+ f):  
            for pic in files:
                p=dir+'/'+f+'/'+pic 
                try:
                    im=Image.open(p)   # if the image opening throws error
                except IOError:
                    count+=1
                    os.remove(p)       # catch it and delete it
            i += 1

filter_images('train_images_model')
filter_images('validation_images_model')
filter_images('test_images_from_train')

In [ ]:
# This cell counts the total number of train images and validation images in the entire dataset
# It stores the counts of train data 108181 training samples belonging to 1000 classes and 
# 19826 test samples in the respective variables nb_train_samples and nb_validation_samples



train_data_dir = 'train_images_model'                   # points to directory having train images
validation_data_dir = 'validation_images_model'         # points to directory having validation images


def count(dir):
    i = 3000
    count = []
    while i <= 4999:
        f = str(i)
        #print (f)
        for root, dirs, files in os.walk(dir +'/'+ f):  # os. walk() is used to iterate through all folders of a directory
            for pic in files:
                count.append(f)   
            i += 1
    print (len(count))
    return ([len(count),count])

nb_train_samples = count(train_data_dir)
nb_validation_samples = count(validation_data_dir)


99724
21400


In [ ]:
#Importing few other files needed
import pandas as pd
import os
import shutil
from shutil import copyfile
import urllib

In [ ]:
# here we build the VGG 16 model, which is pretrained using the imagenet dataset,
# This is achieved by setting weights = 'imagenet' as a parameter while building the model

img_width, img_height = 96, 96 #dimensions used in the model
top_model_weights_path = 'bottleneck_fc_model.h5' # this is used to save weights in later stages, avoid recomputations
epochs = 5
batch_size = 428 #GCD
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range = 0.5,
                                 brightness_range = [0.5,1.5])

    # VGG 16 FramerWork
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(96,96,3))
    print ('start1')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, 
        shuffle=False)
    
    print ('start2')
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples[0] // batch_size) 
    print ('bottleneck_features_trained')
    
    
    with open('bottleneck_features_train.npy', 'wb') as features_train_file:
        np.save(features_train_file, bottleneck_features_train)
    print ('Train done')
    

    datagen = ImageDataGenerator(rescale=1. / 255) 
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print ('validation predict start')
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples[0] // batch_size)
    
    with open('bottleneck_features_validation.npy', 'wb') as features_validation_file:
        np.save(features_validation_file, bottleneck_features_validation)
    print ('validation done')
save_bottleneck_features()

58892288/58889256 [==============================] - 2s 0us/step
start1
Found 99724 images belonging to 2000 classes.
start2
Instructions for updating:
Please use Model.predict, which supports generators.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


bottleneck_features_trained
Train done
Found 21400 images belonging to 2000 classes.
validation predict start
validation done


In [ ]:
# counts of train and validation labels
train_labels = np.array(nb_train_samples[1])
train_labels = [str(int(train_label) - 3000) for train_label in train_labels] 
print(len(train_labels))
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
print(len(train_data))
validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
print(len(validation_data))
validation_labels = np.array(nb_validation_samples[1])
print(len(validation_labels))

99724
99724
21400
21400


In [ ]:
epochs = 5
batch_size = 428
import numpy as np

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(nb_train_samples[1])
    train_labels = [str(int(train_label) - 3000) for train_label in train_labels] 
    # we do label-1000, so that class lable starts from 0, as we have labels starting from 1000

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(nb_validation_samples[1])
    validation_labels = [str(int(validation_label) - 3000) for validation_label in validation_labels]
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    n_class = 2000 # max. classes given to the model
    model.add(Dense(n_class, activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels = to_categorical(train_labels, n_class)
    validation_labels = to_categorical(validation_labels, n_class)
    
    print ('model fit starting')
    
    model.fit(train_data, train_labels,epochs=epochs,batch_size=batch_size,validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

model fit starting
Epoch 1/5
233/233 [==============================] - 4s 17ms/step - loss: 5.1553 - accuracy: 0.1441 - val_loss: 4.3747 - val_accuracy: 0.2388
Epoch 2/5
233/233 [==============================] - 4s 15ms/step - loss: 4.2001 - accuracy: 0.2429 - val_loss: 3.8240 - val_accuracy: 0.3244
Epoch 3/5
233/233 [==============================] - 4s 15ms/step - loss: 3.7311 - accuracy: 0.2988 - val_loss: 3.6022 - val_accuracy: 0.3578
Epoch 4/5
233/233 [==============================] - 4s 15ms/step - loss: 3.4175 - accuracy: 0.3392 - val_loss: 3.2435 - val_accuracy: 0.4210
Epoch 5/5
233/233 [==============================] - 4s 15ms/step - loss: 3.1730 - accuracy: 0.3724 - val_loss: 3.3218 - val_accuracy: 0.4196


In [ ]:
base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(96,96,3))
print(base_model.input)

Tensor("input_2:0", shape=(None, 96, 96, 3), dtype=float32)


In [ ]:
# This is where we fine tune the pretrained model according to our dataset

img_width, img_height = 96, 96
top_model_weights_path = 'bottleneck_fc_model.h5' 
train_data_dir = 'train_images_model'
validation_data_dir = 'validation_images_model'
batch_size = 200
epochs = 50
def trainCNN():

    # build the VGG16 network

    base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(96,96,3))

    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(256, activation='relu'))
    n_class = 2000
    top_model.add(Dense(n_class, activation='softmax'))
    top_model.load_weights(top_model_weights_path) #Load the weights initialized in previous steps
    

    model = Model(base_model.input, top_model(base_model.output))
    
    # set the first 16 layers to non-trainable (weights will not be updated) 
    # 1 conv layer and three dense layers will be trained
    for layer in model.layers[:16]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9,beta_2=0.999,epsilon=1e-8, decay=0.0),
                  metrics=['accuracy'])
    print ('Compilation done.')
    
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=90,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range = 0.5)
    
    valid_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    np.save('class_indices.npy', train_generator.class_indices)

    validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    print ('Model fit begins...')
    model.fit_generator(
        train_generator,
        steps_per_epoch=340,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=100,
        callbacks=[ModelCheckpoint(filepath=top_model_weights_path, save_best_only=True, save_weights_only=True)]
        )
    
    model.save_weights(top_model_weights_path)
    # final weights are saved in bottleneck_fc_model.h5 file

trainCNN()

Compilation done.
Found 99724 images belonging to 2000 classes.
Found 21400 images belonging to 2000 classes.
Model fit begins...
Instructions for updating:
Please use Model.fit, which supports generators.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/50
340/340 [==============================] - 450s 1s/step - loss: 5.0191 - accuracy: 0.1338 - val_loss: 4.4888 - val_accuracy: 0.2026
Epoch 2/50
340/340 [==============================] - 411s 1s/step - loss: 4.1678 - accuracy: 0.2232 - val_loss: 3.9779 - val_accuracy: 0.2918
Epoch 3/50
340/340 [==============================] - 409s 1s/step - loss: 3.8014 - accuracy: 0.2725 - val_loss: 3.7460 - val_accuracy: 0.3413
Epoch 4/50
340/340 [==============================] - 403s 1s/step - loss: 3.5632 - accuracy: 0.3087 - val_loss: 3.5531 - val_accuracy: 0.3729
Epoch 5/50
340/340 [==============================] - 394s 1s/step - loss: 3.3948 - accuracy: 0.3369 - val_loss: 3.4336 - val_accuracy: 0.3968
Epoch 6/50
340/340 [==============================] - 394s 1s/step - loss: 3.2673 - accuracy: 0.3519 - val_loss: 3.2745 - val_accuracy: 0.4268
Epoch 7/50
340/340 [==============================] - 390s 1s/step - loss: 3.1504 - accuracy: 0.3697 - val_loss: 3.0824 - val_accuracy: 0.4572

In [ ]:
from tensorflow.python.platform import app
import argparse
import os
import sys
import time
from time import *
import io
import tensorflow as tf 

In [ ]:
ls

admset213.zip                       test.csv
bottleneck_fc_model.h5              test_images_from_train/
bottleneck_features_train.npy       test_sample.csv
bottleneck_features_validation.npy  train.csv
class_indices.npy                   train_images_model/
result.csv                          train_sample.csv
RUN/                                validation_images_model/
run2.ipynb                          validation_sample.csv


In [ ]:
top_model_weights_path = 'bottleneck_fc_model.h5'       # final weights of the model
train_data_dir = 'train_images_model'
testfile = 'test_images_from_train'
subfile = 'result.csv'                                  # predictions of the model

In [ ]:
from keras import backend as K 
import cv2
top_model_weights_path = 'bottleneck_fc_model.h5'       # final weights of the model
train_data_dir = 'train_images_model'
testfile = 'test_images_from_train'
subfile = 'result2.csv'                                  # predictions of the model
def predict(image_path):
    print ('starting')
    path, dirs, files = next(os.walk(image_path))
    file_len = len(files)
    print('Number of Testimages:', file_len)
    #train_datagen = ImageDataGenerator(rescale=1. / 255)
    #generator = train_datagen.flow_from_directory(train_data_dir, batch_size=batch_size)
    #label_map = (generator.class_indices)
    label_map={}
    for i in range(2000):
      label_map[str(3000+i)]=i
    n_class = 2000
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(n_class, activation='softmax'))
    model = Model(base_model.input,top_model(base_model.output))
    model.load_weights(top_model_weights_path)
    with open(subfile, 'w') as csvfile:
        newFileWriter = csv.writer(csvfile)
        newFileWriter.writerow(['id', 'landmarks','confidence'])
        file_counter = 0
        for root, dirs, files in os.walk(image_path):  # loop through startfolders
            i=0
            for pic in files:
                i+=1
                #loop folder and convert image
                path = image_path + '/' + pic


                orig = cv2.imread(path)
                image = load_img(path, target_size=(96, 96))
                image = img_to_array(image)

                # important! otherwise the predictions will be '0'
                image = image / 255

                image = np.expand_dims(image, axis=0)

                #classify landmark
                

                prediction = model.predict(image)


                class_predicted = prediction.argmax(axis=1)
                #class_predicted = np.argmax(prediction,axis=1)
                #print (pic, class_predicted)


                inID = class_predicted[0]
                #print inID

                inv_map = {v: k for k, v in label_map.items()}
                #print class_dictionary
                label = inv_map[inID]
                score = max(prediction[0])
                scor = "{:.2f}".format(score)
                #out = str(label) + ' '+ scor
                print (i,score,label)
                newFileWriter.writerow([os.path.splitext(pic)[0], int(label),scor])

predict(testfile)

starting
Number of Testimages: 5168
1 0.806817 3274
2 0.38527328 3868
3 0.70156777 3130
4 0.20947924 4124
5 0.3060793 3619
6 0.43088424 4644
7 0.9930876 4352
8 0.19401854 3563
9 0.38142994 4645
10 1.0 3518
11 0.99997365 4352
12 0.7042313 3459
13 0.3953463 3770
14 0.32648245 3924
15 0.93806136 3249
16 0.67641515 3130
17 0.9309056 4352
18 0.39596814 3804
19 0.1562511 4124
20 0.24131607 3518
21 0.3486548 4987
22 0.9271772 4869
23 0.74781805 4352
24 0.17166659 3924
25 0.9999989 3410
26 0.998386 3987
27 0.22848599 3849
28 0.97179776 4873
29 0.4345432 3130
30 0.71738034 3924
31 0.27359554 4388
32 0.7881216 4210
33 0.23062392 4240
34 0.68312335 3332
35 0.35648656 3872
36 0.9986785 4352
37 0.30351555 4987
38 0.22536306 3924
39 0.83417654 4115
40 0.19560361 4737
41 0.37565538 4340
42 0.9993303 3829
43 0.9462506 4352
44 0.3475381 3096
45 0.58318764 3039
46 0.45394483 4493
47 0.37547228 3228
48 0.16636014 4981
49 0.9943416 4353
50 0.54943645 3144
51 0.9983333 4352
52 0.9999883 4352
53 0.99891794 

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Streaming output truncated to the last 5000 lines.
170 0.36638626 3409
171 0.9990102 3497
172 0.20788278 4706
173 0.32107264 3338
174 0.092149645 3455
175 0.8824348 3020
176 0.348174 3931
177 0.62602293 3561
178 0.9958187 3151
179 0.26406842 4954
180 0.5272394 3397
181 0.36068127 4470
182 0.06856489 4070
183 0.40414912 3372
184 0.96442586 4352
185 0.25304294 4776
186 0.19494724 4987
187 0.62533355 4330
188 0.9942913 3283
189 0.9999988 3804
190 0.4406023 4170
191 0.25130677 3296
192 0.99492973 4190
193 0.47623056 4275
194 0.9941241 3804
195 0.46039718 4830
196 0.8896221 4934
197 0.9815957 4327
198 0.99999213 4105
199 0.47569606 4010
200 0.183693 3924
201 0.95186466 3518
202 0.4375229 3987
203 0.90219325 4624
204 0.8207762 4028
205 0.1470564 4686
206 0.59606713 4957
207 0.40412012 3924
208 0.3677414 3128
209 0.058572818 3962
210 0.9663109 4954
211 0.55696166 3749
212 0.2588967 3664
213 0.113977745 3596
214 0.98530895 4085
215 0.4769343 4470
216 0.116282105 4330
217 0.9999747 4108
218 0.3

In [ ]:
#result = pd.read_csv('result.csv')     # predicted values

In [ ]:
#test = pd.read_csv('test_sample.csv')  # ground truth

In [ ]:
# final accuracy of the model as tested on test dataset
import pandas as pd
def test_accuracy(pred,truth):
    result=pd.read_csv(pred)
    #print(result.head(20))
    test=pd.read_csv(truth)
    count=0
    for i in result["id"]:
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #b=list(b[0].split(" "))
        b[0]=int(b[0])
        if i in test.id.values:
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              count+=1
    print(count/len(result))
    return count/len(result)
test_accuracy("result2.csv","test_sample.csv")

0.6106811145510835


0.6106811145510835

In [ ]:
# This call calulates the Global Average Precision Metric 
# 
import pandas as pd
def GAP_metric(path):
    result=pd.read_csv(path)
    test=pd.read_csv("test_sample.csv")
    result=result.sort_values(by='confidence', ascending=False)
    M=0
    true=0
    total=0
    s=0
    relevance=0
    for i in result['id']:
        total+=1
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #if i in test['id']:
        if i in test.id.values:
          M+=1
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              true+=1
              relevance=1
        s+=(true/total)*relevance
        relevance=0
    print(M,s,total,true)
    print(s/M)
    return s/M
GAP_metric("result2.csv")

5168 2947.5271513731795 5168 3156
0.5703419410551818


0.5703419410551818

In [ ]:
#GAP_metric('result.csv')